## Суть проекта 

Отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

ds = pd.read_csv('stud_math.xls')


len(ds.columns)
# Ответ 30 - какой-то столбец лишний по отношению к начальным данным. Посмотрев названия столбцов, удаляем лишний.
ds.drop(['studytime, granular'], inplace=True, axis=1)
# Оценим числовые данные в целом
ds.describe()
# Ошибка в famrel, она не может быть -1, ошибка в Fedu, не может быть больше 4. Что-то не так с abscence,
# большое максимальное значение при маленьком 75% квантиле.


# Функция, которая считает количество уникальных значений в столбце.
# Оценим с ее помощью частоту появления данных в столбцах
def data_glance(dataset):
    for i in range(len(dataset.columns)):
        dataset[dataset.columns[i]].value_counts(
            dropna=False).plot(kind='barh')
        plt.legend()
        plt.show()


data_glance(ds)
# Столбцы,не содержащие nan: school, sex, age
# Ошибки в Fedu (одно из значений 40), famrel (одно из значений -1)


# Функция,отсеивающая выбросы из массива, вдруг пригодится.

def anomaly_exclude_hist(column):
    median = column.median()
    IQR = column.quantile(0.75)-column.quantile(0.25)
    perc25 = column.quantile(0.25)
    perc75 = column.quantile(0.75)
    column.loc[column.between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)
               ].plot.hist(bins=10, range=(0, perc75 + 1.5*IQR))
    plt.legend()

# Не уверен, что нужно удалять выбросы из столбца abscence, всё-таки можно и две недели пропустить, и полгода, всякое бывает.
#abscence_hist = anomaly_exclude_hist(ds.absences)


# Удаляю из всех столбцов все ошибки.В образовательных целях предположим, что я исправил верно :3
ds.Fedu = ds.Fedu.replace(40, 4)
ds.famrel = ds.famrel.replace(-1, 1)


# Функция, строящая графики зависимости финального балла от номинативных переменных

def get_boxplot(column):
    fig, ax = plt.subplots(figsize=(4, 4))
    sns.boxplot(x=column, y='score',
                data=ds, ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

    
# Функция, применяющая get_boxplot ко всем столбцам с номинативными переменными

def data_glance_cat(dataset):
    for i in range(len(dataset.columns)):
        if ds[ds.columns[i]].dtypes not in ('int64', 'float64'):
            get_boxplot(ds.columns[i])


# Посмотрим корреляцию между финальным баллом и количественными переменными.
display(ds.corrwith(ds.score).sort_values())
'''Удивительно, самая большая корреляция с образованием родителей и временем за учебой. 
Кто бы мог подумать, что умные и старательные дети преуспевают на экзаменах лучше.
А быть неудачником, это серьезно. 
Хотя несколько удивляет наличие некоторой отрицательной корреляции с возрастом.'''
# Не коррелирующие переменные (модуль коэффициента корреляции < 0.1): traveltime,health,freetime,famrel,abscences.

# Аналогично для номинальных переменных
data_glance_cat(ds)
'''Похоже, значимые переменные тут это Mjob, Fjob, schoolsup,higher,internet. 
Что странно - наличие дополнительной образовательной поддержки отрицательно сказывается на финальном балле.'''

#Функция для нахождения статистической значимости для номинативных переменных (я не понял, как она работает, помогите плз в слаке).
def get_stat_dif(column):
    cols = ds.loc[:, column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(ds.loc[ds.loc[:, column] == comb[0], 'score'],
                     ds.loc[ds.loc[:, column] == comb[1], 'score']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

#Применяю эту функцию к столбцам с номинативными переменными. Можно было указать врукопашную только нужные столбцы,но данных не так много.
def data_glance_cat2(dataset):
    for i in range(len(dataset.columns)):
        if ds[ds.columns[i]].dtypes not in ('int64', 'float64'):
            get_stat_dif(ds.columns[i])


data_glance_cat2(ds)
# Статистически значимые различия только для Mjob
'''Таким образом данные, которые можно будет использовать в дальнейшем построении модели:
traveltime,health,freetime,famrel,abscences,Mjob
Данные очищены от ошибок.'''

ds_for_model = ds.loc[:, ['Mjob', 'Medu', 'Fedu',
                          'studytime', 'goout', 'age', 'failures', 'score']]
ds_for_model.head()

FileNotFoundError: [Errno 2] File stud_math.xls does not exist: 'stud_math.xls'